In [7]:
import pybullet as p
import pybullet_data
import time
import numpy as np
import math
from PIL import Image

def cal_xy(theta, b, a, r=0.15):
    theta = (theta / 3.1416) * 180 #()
    #y = b - abs(math.sin(math.radians(theta+270)))*r
    y = b - abs(math.cos(math.radians(theta)))*r
    x = a - math.sin(math.radians(theta))*r

    print(theta,a, x, y)
    return x*1.333, y

physicsClient = p.connect(p.GUI) # GUI Window
p.setAdditionalSearchPath(pybullet_data.getDataPath()) # data path
p.resetSimulation() 
p.setGravity(0, 0, 0) # set gravity

# camera setting
view_matrix1=p.computeViewMatrixFromYawPitchRoll(cameraTargetPosition=[0.5,0,1.5],
                                                    distance=.1,
                                                    yaw=-90,
                                                    pitch=-60,
                                                    roll=0,upAxisIndex=2)
proj_matrix1=p.computeProjectionMatrixFOV(fov=69,
                                            aspect=1.0,
                                            nearVal=0.1,
                                            farVal=100.0)


#obj 
planeID = p.loadURDF("plane.urdf")
#tableUid = p.loadURDF("table/table.urdf")
robotId = p.loadSDF("kuka_iiwa/model.sdf")
cylinderId = p.loadURDF("table/table.urdf")
boxId = p.loadURDF("objects/mug.urdf", globalScaling=1.2)
#boxId = p.loadURDF("duck_vhacd.urdf", globalScaling=2)


#obj pos and orn setting
robotStartPos = [0, 0, 0.65]
robotStartOrn = p.getQuaternionFromEuler([0,0,0])
p.resetBasePositionAndOrientation(robotId[0],robotStartPos,robotStartOrn)

image_path = 'img9/%s.png'%('%s')
boxStartPos = [0.9, -0.3, 0.58]
boxStartOrn = [0,0,-0.7584]
p.resetBasePositionAndOrientation(boxId,boxStartPos,p.getQuaternionFromEuler(boxStartOrn))

cylinderStartPos = [0.8, 0, -0.05]
cylinderStartOrn = p.getQuaternionFromEuler([0,0,0])
p.resetBasePositionAndOrientation(cylinderId,cylinderStartPos,cylinderStartOrn)

# cal robot move plan
p.getNumJoints(robotId[0]) # get joint num
p.getJointInfo(robotId[0], 6) # get joint inf
robotStartPos = [0.65, 0, 0.65]
robotStartOrn = [1.5708, 3.1416, 1.5708]
objX, objY = cal_xy(boxStartOrn[2], boxStartPos[0], boxStartPos[1])
# robotEndPos = [(boxStartPos[0]-robotStartPos[0])*ratio+robotStartPos[0], (boxStartPos[1]-robotStartPos[1])*ratio+robotStartPos[1], 0.625]
robotEndPos = [objY, objX, 0.65]
robotEndOrn = [1.5708, 0, boxStartOrn[2]+1.5708]

startPos_array = np.array(robotStartPos)
endPos_array = np.array(robotEndPos)
startOrn_array = np.array(robotStartOrn)
endOrn_array = np.array(robotEndOrn)

targetPositionsJoints = p.calculateInverseKinematics(robotId[0], 6, startPos_array,\
    targetOrientation=p.getQuaternionFromEuler(startOrn_array))# IK
p.setJointMotorControlArray(robotId[0], range(7), p.POSITION_CONTROL,\
         targetPositions=targetPositionsJoints) # move plan

stepNum = 50
stepPos_array = (endPos_array - startPos_array) / stepNum
StepOrn_array = (endOrn_array - startOrn_array) / stepNum

for i in range(stepNum+1):
    #print(i, "step")
    robotStepPos = list(startPos_array) # next position
    robotStepOrn = list(startOrn_array)
    targetPositionsJoints = p.calculateInverseKinematics(robotId[0], 6, robotStepPos,\
        targetOrientation=p.getQuaternionFromEuler(robotStepOrn))# IK
    p.setJointMotorControlArray(robotId[0], range(7), p.POSITION_CONTROL,\
         targetPositions=targetPositionsJoints) # move plan
    # for i in range(10): #time
    p.stepSimulation()
    time.sleep(1/240)
for i in range(stepNum+1):
    #print(i, "step")
    robotStepPos = list(stepPos_array*i + startPos_array) # next position
    robotStepOrn = list(StepOrn_array*i + startOrn_array)
    targetPositionsJoints = p.calculateInverseKinematics(robotId[0], 6, robotStepPos,\
        targetOrientation=p.getQuaternionFromEuler(robotStepOrn))# IK
    p.setJointMotorControlArray(robotId[0], range(7), p.POSITION_CONTROL,\
         targetPositions=targetPositionsJoints) # move plan
    # for i in range(10): #time
    p.stepSimulation()
    time.sleep(1/10)
    (_,_,px1,_,_) = p.getCameraImage(width=200,height=200,
                            viewMatrix=view_matrix1,
                            projectionMatrix=proj_matrix1,
                            renderer=p.ER_BULLET_HARDWARE_OPENGL)
    rgb_array1=np.array(px1,dtype=np.uint8)
    rgb_array1=np.reshape(rgb_array1,(200,200,4))
    rgb_array1=rgb_array1[:,:,:3]
    rgb_array1 = Image.fromarray(rgb_array1)
    img_dir = image_path%(str(i).zfill(3))
    rgb_array1.save(img_dir)
    #print(robotStepPos, robotStepOrn)
    #print("---------")
    # startPos_array = np.array(robotStepPos)
    # startOrn_array = np.array(robotStepOrn)
p.disconnect()

-43.45301757066463 -0.3 -0.1968360691884926 0.7911092135232806


In [3]:
p.disconnect()